In [ ]:
import datasets


import tensorflow_datasets as tfds
config = tfds.translate.wmt.WmtConfig(
    version="0.0.1",
    language_pair=("fr", "de"),
    subsets={
        tfds.Split.TRAIN: ["commoncrawl_frde"],
        tfds.Split.VALIDATION: ["euelections_dev2019"],
    },
)
builder = tfds.builder("wmt_translate", config=config)

In [ ]:
from datasets import load_dataset

dataset = load_dataset('xsum', 'test')

In [ ]:
d = dataset['validation']
cnt = 500
for ex in d:
    print(ex['summary'])
    break

In [ ]:
test_set = dataset['test']
print(test_set[0])

In [ ]:
# T5 translation
from transformers import AutoTokenizer, AutoModelWithLMHead
  
tokenizer = AutoTokenizer.from_pretrained("t5-base")

model = AutoModelWithLMHead.from_pretrained("t5-base")

In [ ]:
# facebook mbart
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
article_en = "The head of the United Nations says there is no military solution in Syria"
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")

model_inputs = tokenizer(article_en, return_tensors="pt")

# translate from English to Chinese
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["ja_XX"]
)
print(generated_tokens)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => '联合国首脑说,叙利亚没有军事解决办法'

In [ ]:
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["fr_XX"],num_return_sequences=5,num_beams=1,top_p=0.99, do_sample=True
)
# print(generated_tokens)
print(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True))

In [ ]:
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["zh_CN"],num_return_sequences=5,num_beams=1,top_p=0.5, do_sample=True
)
# print(generated_tokens)
print(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True))

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

article_hi = "संयुक्त राष्ट्र के प्रमुख का कहना है कि सीरिया में कोई सैन्य समाधान नहीं है"
article_ar = "الأمين العام للأمم المتحدة يقول إنه لا يوجد حل عسكري في سوريا."
article_zh = "全球主要经济体的领导人同意采取“有意义和有效的行动”，努力控制全球变暖。但是，在罗马举行的G20峰会上达成的协议几乎没有做出具体承诺，这让活动人士感到失望。主办国意大利曾希望在格拉斯哥召开的COP26峰会之前设定明确目标，而这次峰会现在已经开始了。"

# model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")


In [ ]:
x = 10740
print(tokenizer.decode([x]))
print(tokenizer.convert_ids_to_tokens(x))
print(tokenizer.eos_token_id)

In [ ]:
print(tokenizer.special_tokens_map)
print(tokenizer.lang_code_to_id)
print(tokenizer.special_tokens_map['additional_special_tokens'])
# translate Hindi to English
tokenizer.src_lang = "hi_IN"
encoded_hi = tokenizer(article_hi, return_tensors="pt")
generated_tokens = model.generate(**encoded_hi)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "The head of the UN says there is no military solution in Syria."
print(generated_tokens)
# print(tokenizer.lang_code_to_id("hi_IN"))
# translate Arabic to English
tokenizer.src_lang = "ar_AR"
encoded_ar = tokenizer(article_ar, return_tensors="pt")
generated_tokens = model.generate(**encoded_ar)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "The Secretary-General of the United Nations says there is no military solution in Syria."
print(generated_tokens)


# translate Arabic to English
tokenizer.src_lang = "zh_CN"
encoded_ar = tokenizer(article_zh, return_tensors="pt")
generated_tokens = model.generate(**encoded_ar)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(generated_tokens)

# => "The Secretary-General of the United Nations says there is no military solution in Syria."



In [ ]:
print(tokenizer.decode(2))

In [ ]:

# translate Hindi to English
article_hi = "संयुक्त राष्ट्र के प्रमुख का कहना है कि सीरिया में कोई सैन्य समाधान नहीं है"
article_zh="为什么是这？谁知道"
# translate Arabic to English
tokenizer.src_lang = "zh_CN"
encoded_ar = tokenizer(article_zh, return_tensors="pt")
generated_tokens = model.generate(**encoded_ar)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(generated_tokens)

print(tokenizer.decode([2,250010,44269],skip_special_tokens=False)) 
print(tokenizer.decode([2, 250025,6],skip_special_tokens=False))

In [ ]:
from datasets import load_metric
from sacrebleu.metrics import BLEU, CHRF, TER

In [ ]:
bleu = BLEU()
sys = ['今天 是 个 好 日子']
refs=[['今天 天气 好', '今天 很 不错']]
print(bleu.corpus_score(sys, refs))
bleu.get_signature()
chrf = CHRF()

chrf.corpus_score(sys, refs)

In [ ]:
with open('/home/jcxu/.sacrebleu/wmt17/en-zh.zh', 'r') as fd:
    lines = fd.read().splitlines()
print(lines[:5])

# with open('/home/jcxu/.sacrebleu/wmt17/en-zh.en', 'r') as fd:
#     lines = fd.read().splitlines()
# print(lines[:5])

import sacrebleu
from sacremoses import MosesDetokenizer
md = MosesDetokenizer(lang='zh')
md.detokenize(lines[0])